In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv('Datasets/customer_behavior_dataset.csv')
dataset.head()

,Age,Gender,Income (in $1000s),Online Hours Per Week,Purchase Frequency (Last Month),Is Loyal Customer,Preferred Shopping Platform,Ad Clicks (Last Month),Discounts Used (Last Month),Competitor Pricing Index,Economic Conditions,Distance to Nearest Store (km),Shopping Frequency,"Purchase Decision (1=Yes, 0=No)"
0,56,Female,91.97,49,7,0,Website,1,9,0.83,Neutral,43.09,1.685360,1
1,69,Male,22.29,1,2,1,Website,9,3,1.13,Neutral,16.78,15.337143,1
2,46,Female,50.59,3,7,0,Website,0,2,1.06,Favorable,9.80,17.530050,1
3,32,Male,58.63,49,14,1,In-store,0,8,0.94,Neutral,2.78,28.540342,1
4,60,Female,44.87,27,0,0,Mobile App,7,9,0.87,Unfavorable,49.22,1.000000,1


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Age                              5000 non-null   int64  
 1   Gender                           5000 non-null   object 
 2   Income (in $1000s)               4800 non-null   float64
 3   Online Hours Per Week            5000 non-null   int64  
 4   Purchase Frequency (Last Month)  5000 non-null   int64  
 5   Is Loyal Customer                5000 non-null   int64  
 6   Preferred Shopping Platform      5000 non-null   object 
 7   Ad Clicks (Last Month)           5000 non-null   int64  
 8   Discounts Used (Last Month)      5000 non-null   int64  
 9   Competitor Pricing Index         5000 non-null   float64
 10  Economic Conditions              4850 non-null   object 
 11  Distance to Nearest Store (km)   5000 non-null   float64
 12  Shopping Frequency  

In [4]:
# Since missing values are low dropping is best opion
dataset = dataset.dropna()
dataset.shape

(4660, 14)

In [5]:
categorical_features = ['Gender','Preferred Shopping Platform', 'Economic Conditions']
for f in categorical_features:
    print(f'{f}: {dataset[f].unique()}')

Gender: ['Female' 'Male']
Preferred Shopping Platform: ['Website' 'In-store' 'Mobile App']
Economic Conditions: ['Neutral' 'Favorable' 'Unfavorable']


In [7]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)
one_hot_encoded = ohe.fit_transform(dataset[['Gender']])
ohe_hot_encoded_df = pd.DataFrame(one_hot_encoded, columns=ohe.get_feature_names_out(['Gender']))
le = LabelEncoder()
encoded_dataset1 = dataset.copy()
for f in categorical_features:
    if f != 'Gender':  
        encoded_dataset1[f] = le.fit_transform(encoded_dataset1[f])

encoded_dataset = pd.concat(
    [encoded_dataset1.drop(columns=['Gender'], axis=1).reset_index(drop=True), ohe_hot_encoded_df.reset_index(drop=True)], axis=1)

In [9]:
X, y = encoded_dataset.drop(columns=['Purchase Decision (1=Yes, 0=No)'], axis=1).values, encoded_dataset['Purchase Decision (1=Yes, 0=No)'].values
X, y

(array([[56.        , 91.97      , 49.        , ...,  1.68536028,
          1.        ,  0.        ],
        [69.        , 22.29      ,  1.        , ..., 15.33714311,
          0.        ,  1.        ],
        [46.        , 50.59      ,  3.        , ..., 17.53005006,
          1.        ,  0.        ],
        ...,
        [26.        , 53.61      ,  9.        , ...,  1.        ,
          0.        ,  1.        ],
        [53.        , 42.04      ,  5.        , ..., 16.73424535,
          1.        ,  0.        ],
        [36.        , 55.16      ,  6.        , ..., 11.57171689,
          1.        ,  0.        ]], shape=(4660, 14)),
 array([1, 1, 1, ..., 1, 1, 1], shape=(4660,)))

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train_scaled, X_test_scaled = scaler.fit_transform(X_train), scaler.transform(X_test)
X_train_scaled, X_test_scaled, y_train, y_test

(array([[-0.37682941,  1.2497109 , -0.99712663, ...,  1.77213258,
         -0.99145283,  0.99145283],
        [ 0.75095395,  0.50765692, -0.92815366, ...,  1.80100262,
          1.00862085, -1.00862085],
        [ 1.34801573, -0.42815006,  0.17541377, ...,  0.55682696,
         -0.99145283,  0.99145283],
        ...,
        [-0.31048921,  0.4192696 , -0.03150512, ...,  1.31166512,
         -0.99145283,  0.99145283],
        [ 0.35291277,  0.18756499,  1.55487306, ...,  1.64263582,
          1.00862085, -1.00862085],
        [-1.04023138,  0.42725941,  0.45130563, ...,  0.78783563,
         -0.99145283,  0.99145283]], shape=(3262, 14)),
 array([[ 1.14899514, -0.77570697,  0.45130563, ...,  1.54416   ,
         -0.99145283,  0.99145283],
        [ 0.35291277,  0.90964444,  0.79617045, ..., -1.56080825,
          1.00862085, -1.00862085],
        [ 0.81729415,  0.11915221, -1.06609959, ...,  0.14621527,
         -0.99145283,  0.99145283],
        ...,
        [ 1.28167553,  0.68592964,  

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
logiregressor = LogisticRegression()
logiregressor.fit(X_train_scaled, y_train)
y_pred=logiregressor.predict(X_test)
print(f'accuracy: {accuracy_score(y_pred,y_test)}')
print(classification_report(y_pred,y_test))
print(confusion_matrix(y_pred,y_test))

accuracy: 0.9907010014306151
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.99      1.00      1398

    accuracy                           0.99      1398
   macro avg       0.50      0.50      0.50      1398
weighted avg       1.00      0.99      1.00      1398

[[   0    0]
 [  13 1385]]


c:\Users\bvkir\OneDrive\Desktop\Machine learning\mlenv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\bvkir\OneDrive\Desktop\Machine learning\mlenv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\bvkir\OneDrive\Desktop\Machine learning\mlenv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita

# Lets try without dropping NAN values instead adding mode or mean

In [12]:
dataset = pd.read_csv('Datasets/customer_behavior_dataset.csv')
dataset.head()

,Age,Gender,Income (in $1000s),Online Hours Per Week,Purchase Frequency (Last Month),Is Loyal Customer,Preferred Shopping Platform,Ad Clicks (Last Month),Discounts Used (Last Month),Competitor Pricing Index,Economic Conditions,Distance to Nearest Store (km),Shopping Frequency,"Purchase Decision (1=Yes, 0=No)"
0,56,Female,91.97,49,7,0,Website,1,9,0.83,Neutral,43.09,1.685360,1
1,69,Male,22.29,1,2,1,Website,9,3,1.13,Neutral,16.78,15.337143,1
2,46,Female,50.59,3,7,0,Website,0,2,1.06,Favorable,9.80,17.530050,1
3,32,Male,58.63,49,14,1,In-store,0,8,0.94,Neutral,2.78,28.540342,1
4,60,Female,44.87,27,0,0,Mobile App,7,9,0.87,Unfavorable,49.22,1.000000,1


In [15]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Age                              5000 non-null   int64  
 1   Gender                           5000 non-null   object 
 2   Income (in $1000s)               4800 non-null   float64
 3   Online Hours Per Week            5000 non-null   int64  
 4   Purchase Frequency (Last Month)  5000 non-null   int64  
 5   Is Loyal Customer                5000 non-null   int64  
 6   Preferred Shopping Platform      5000 non-null   object 
 7   Ad Clicks (Last Month)           5000 non-null   int64  
 8   Discounts Used (Last Month)      5000 non-null   int64  
 9   Competitor Pricing Index         5000 non-null   float64
 10  Economic Conditions              4850 non-null   object 
 11  Distance to Nearest Store (km)   5000 non-null   float64
 12  Shopping Frequency  

In [16]:
dataset['Income (in $1000s)'] = dataset['Income (in $1000s)'].fillna(dataset['Income (in $1000s)'].mean())
dataset['Economic Conditions'] = dataset['Economic Conditions'].fillna(dataset['Economic Conditions'].mode()[0])
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Age                              5000 non-null   int64  
 1   Gender                           5000 non-null   object 
 2   Income (in $1000s)               5000 non-null   float64
 3   Online Hours Per Week            5000 non-null   int64  
 4   Purchase Frequency (Last Month)  5000 non-null   int64  
 5   Is Loyal Customer                5000 non-null   int64  
 6   Preferred Shopping Platform      5000 non-null   object 
 7   Ad Clicks (Last Month)           5000 non-null   int64  
 8   Discounts Used (Last Month)      5000 non-null   int64  
 9   Competitor Pricing Index         5000 non-null   float64
 10  Economic Conditions              5000 non-null   object 
 11  Distance to Nearest Store (km)   5000 non-null   float64
 12  Shopping Frequency  

In [17]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)
one_hot_encoded = ohe.fit_transform(dataset[['Gender']])
ohe_hot_encoded_df = pd.DataFrame(one_hot_encoded, columns=ohe.get_feature_names_out(['Gender']))
le = LabelEncoder()
encoded_dataset1 = dataset.copy()
for f in categorical_features:
    if f != 'Gender':  
        encoded_dataset1[f] = le.fit_transform(encoded_dataset1[f])

encoded_dataset = pd.concat(
    [encoded_dataset1.drop(columns=['Gender'], axis=1).reset_index(drop=True), ohe_hot_encoded_df.reset_index(drop=True)], axis=1)

In [18]:
encoded_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Age                              5000 non-null   int64  
 1   Income (in $1000s)               5000 non-null   float64
 2   Online Hours Per Week            5000 non-null   int64  
 3   Purchase Frequency (Last Month)  5000 non-null   int64  
 4   Is Loyal Customer                5000 non-null   int64  
 5   Preferred Shopping Platform      5000 non-null   int64  
 6   Ad Clicks (Last Month)           5000 non-null   int64  
 7   Discounts Used (Last Month)      5000 non-null   int64  
 8   Competitor Pricing Index         5000 non-null   float64
 9   Economic Conditions              5000 non-null   int64  
 10  Distance to Nearest Store (km)   5000 non-null   float64
 11  Shopping Frequency               5000 non-null   float64
 12  Purchase Decision (1

In [19]:
X, y = encoded_dataset.drop(columns=['Purchase Decision (1=Yes, 0=No)'], axis=1).values, encoded_dataset['Purchase Decision (1=Yes, 0=No)'].values
X, y

(array([[56.        , 91.97      , 49.        , ...,  1.68536028,
          1.        ,  0.        ],
        [69.        , 22.29      ,  1.        , ..., 15.33714311,
          0.        ,  1.        ],
        [46.        , 50.59      ,  3.        , ..., 17.53005006,
          1.        ,  0.        ],
        ...,
        [26.        , 53.61      ,  9.        , ...,  1.        ,
          0.        ,  1.        ],
        [53.        , 42.04      ,  5.        , ..., 16.73424535,
          1.        ,  0.        ],
        [36.        , 55.16      ,  6.        , ..., 11.57171689,
          1.        ,  0.        ]], shape=(5000, 14)),
 array([1, 1, 1, ..., 1, 1, 1], shape=(5000,)))

In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train_scaled, X_test_scaled = scaler.fit_transform(X_train), scaler.transform(X_test)
X_train_scaled, X_test_scaled, y_train, y_test

(array([[-0.91132783, -0.69066401, -0.97738846, ...,  0.62440367,
         -0.96628239,  0.96628239],
        [-0.84481864,  1.23577697, -0.42499096, ..., -0.96644206,
          1.03489416, -1.03489416],
        [-1.31038292, -0.84827728,  1.57744997, ..., -0.31426032,
          1.03489416, -1.03489416],
        ...,
        [ 0.48536501, -1.59312712, -0.90833877, ...,  0.34414191,
          1.03489416, -1.03489416],
        [-1.64292884,  2.03757095,  0.61075435, ..., -1.38317761,
         -0.96628239,  0.96628239],
        [ 0.48536501, -1.79599066, -0.07974252, ..., -1.10516458,
          1.03489416, -1.03489416]], shape=(3500, 14)),
 array([[ 1.01743848,  0.53160153, -0.14879221, ...,  1.44056795,
          1.03489416, -1.03489416],
        [-1.24387374, -1.56109603,  1.50840028, ...,  0.55237516,
         -0.96628239,  0.96628239],
        [-0.37925436, -0.03173883, -1.66788533, ...,  0.988138  ,
         -0.96628239,  0.96628239],
        ...,
        [ 0.41885583,  1.4503344 , -

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
logiregressor = LogisticRegression()
logiregressor.fit(X_train_scaled, y_train)
y_pred=logiregressor.predict(X_test)
print(f'accuracy: {accuracy_score(y_pred,y_test)}')
print(classification_report(y_pred,y_test))
print(confusion_matrix(y_pred,y_test))

accuracy: 0.9926666666666667
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.99      1.00      1500

    accuracy                           0.99      1500
   macro avg       0.50      0.50      0.50      1500
weighted avg       1.00      0.99      1.00      1500

[[   0    0]
 [  11 1489]]


c:\Users\bvkir\OneDrive\Desktop\Machine learning\mlenv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\bvkir\OneDrive\Desktop\Machine learning\mlenv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\bvkir\OneDrive\Desktop\Machine learning\mlenv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita